In [ ]:
# ! pip install --upgrade pip
# !pip install matplotlib
# !pip install statsmodels
# ! pip install numpy
# ! pip install pandas
# ! pip install tensorflow-macos
# ! pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt

from models_utils import compile_and_fit
from window_generator import WindowGenerator

Reading data from csv and doing initial manipulations

In [ ]:
sales_data = pd.read_csv('sales_train.csv')
df = sales_data.drop(columns='date_block_num').copy()
df['date'] = pd.to_datetime(sales_data['date'], format="%d.%m.%Y")
df.set_index('date', inplace=True)
df.describe()

Getting the longest series of a (item, shop) pair

In [ ]:
(longest_item_id, longest_shop_id), length = df.groupby(['item_id', 'shop_id']).size().agg(['idxmax','max'])
print(f'{longest_item_id=}\n{longest_shop_id=}\n{length=}')
df: pd.DataFrame = df.copy()[(df.shop_id == longest_shop_id) & (df.item_id == longest_item_id)]
df.drop(columns=['shop_id', 'item_id'], inplace=True)
df.sort_index(inplace=True)
df.head()

Creating a column to store the cumulative sum of sales on each month, this will be the target of the models

In [ ]:
df.loc[:, 'qtt_cumsum'] = df.groupby(lambda d: (d.month, d.year)).item_cnt_day.cumsum()
df

Visualizing the series by year

In [ ]:
df.reset_index(inplace=True)
plot_cols = ['date', 'qtt_cumsum']
plot_features = df.copy().loc[:,plot_cols]
plot_features.loc[:, 'year'] = plot_features.date.dt.year
plot_features.loc[:, 'date'] = plot_features.date.dt.strftime('%m-%d')
unstacked = plot_features.set_index(['year', 'date']).qtt_cumsum.unstack(-2)
_ = unstacked.plot()

### Feature engineering
Converting date of the month and year to sin and cos signals, so that the models can better understand those inputs.

In [ ]:
timestamp_s = df['date'].map(pd.Timestamp.timestamp)
timestamp_s

In [ ]:
month = 30.44 * 24 * 60 * 60
year = 12 * month

df['Month sin'] = np.sin(timestamp_s * (2 * np.pi / month))
df['Month cos'] = np.cos(timestamp_s * (2 * np.pi / month))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
df = df.drop(columns='date')

In [ ]:
plt.plot(np.array(df['Month sin'])[:31])
plt.plot(np.array(df['Month cos'])[:31])
plt.xlabel('Time [d]')
_ = plt.title('Day of month signal')

In [ ]:
df[:30]

Splitting the dataset: 70% for train, 10% for validation and 20% for test.

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

df_size = len(df)
train_df = df.copy()[0:int(df_size*0.70)]
val_df = df.copy()[int(df_size*0.70):int(df_size*.80)]
test_df = df.copy()[int(df_size*.80):]
train_df.describe()

### Item Price
As seeing in the next graph, the price colum have some outliers.

In [ ]:
_ = df.item_price.plot()

So values less than the 10th percentile will be replaced with the 10th percentile

In [ ]:
low_quantile = df.item_price.quantile(.1)
df.item_price < low_quantile
item_price_normed = df.item_price.mask(df.item_price < low_quantile, low_quantile)
item_price_normed.plot()

After the normalization the price became an input with no differentiation: it became 5.0 in the whole serie. Therefore, it will not be considered an informative input to the model.

In [ ]:
df.drop(columns='item_price', inplace=True)
train_df.drop(columns='item_price', inplace=True)
val_df.drop(columns='item_price', inplace=True)
test_df.drop(columns='item_price', inplace=True)

### Normalization
Normalizing the data with the z-score.

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
import seaborn as sns
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

In [ ]:
train_df.describe()

### Multi-step models

First we create a window generator that receive 50 days and will generate outputs for 30 days in the future.

In [ ]:
OUT_STEPS = 30
multi_window = WindowGenerator(input_width=50,
                               label_width=OUT_STEPS,
                               train_df=train_df, val_df=val_df, test_df=test_df,
                               shift=OUT_STEPS, label_columns=['qtt_cumsum']
                               )

multi_window.plot()
multi_window

### Baseline

The baseline only repeats the last value from the input series.

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(last_baseline)

### Single-shot models
The following models try to predict the future in a single step.

#### Linear

In [ ]:
num_features = df.shape[1]

multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_linear_model)

#### Dense

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_dense_model)

#### Convolutional

In [ ]:
num_features = train_df.shape[1]

CONV_WIDTH = 9
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_conv_model, multi_window)

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_conv_model)

#### Recurrent Neural Network

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)


multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

### Autoregressive model

In [ ]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

    def warmup(self, inputs):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state

    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the LSTM state.
        prediction, state = self.warmup(inputs)

        # Insert the first prediction.
        predictions.append(prediction)

        # Run the rest of the prediction steps.
        for n in range(1, self.out_steps):
            # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state,
                                      training=training)
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)
            # Add the prediction to the output.
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)


multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(feedback_model)

### Performance

In [ ]:
x = np.arange(len(multi_val_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = multi_lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_val_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()

In [ ]:
for name, value in multi_performance.items():
  print(f'{name:8s}: {value[1]:0.4f}')

The CNN was the model the had the best performance among the models tested. Lets see some examples of its predictions from the test data.

In [ ]:
example_window = tf.stack([np.array(test_df[3:3+multi_window.total_window_size]),
                           np.array(test_df[34:34+multi_window.total_window_size]),
                           np.array(test_df[-multi_window.total_window_size:])])

multi_window.plot(multi_conv_model, example=multi_window.split_window(example_window), denormalize=(train_mean, train_std))

It's noticeable how the model tends to exacerbate it's predictions. If we take a look at the series plotted on the beggining of this notebook, is its clear how the first two years (2013 and 2014) had greater total sales. This can be pushing the predicted values up.
To enhance the model assertivity, would probably be necessary to use more data, like 3 more years, for instance.